Towards 3D Deep Learning for neuropsychiatry: predicting Autism diagnosis using an interpretable Deep Learning pipeline applied to minimally processed structural MRI data, Melanie Garcia, Clare Kelly. medRxiv 2022.10.18.22281196; doi: https://doi.org/10.1101/2022.10.18.22281196

Github: https://github.com/garciaml/Autism-3D-CNN-brain-sMRI?tab=readme-ov-file

# Data Preprocessing

Stucture your data following the BIDS organization (https://bids.neuroimaging.io/). Your participants.tsv file should contain:

1. a column called participant_id corresponding to the subject id (the same as the folder names in the BIDS dataset: sub-<participant_id>).
2. a column called label corresponding to the binary target variable (here: 0=no diagnosis, 1=Autism)
3. a column called dataset corresponding to where (training, validation, testing set) each participant data will be used: the code supports the three modalities train, val, test.

In [ ]:
# Create ABIDEII_BIDS folder in BIDS structure from ABIDEII_Raw

In [ ]:
# clone github repo from above
#!git clone https://github.com/garciaml/Autism-3D-CNN-brain-sMRI.git
#!cd ../Autism-3D-CNN-brain-sMRI
#!pip install -r requirements.txt
#!python preprocessing_bids.py ABIDEII_BIDS ABIDEII_BIDS_PROCESSED